In [1]:
import pandas as pd
import sys
import os.path
import numpy as np
import cProfile
import pstats
import matplotlib.pyplot as plt
# import bokeh.plotting
# from bokeh.plotting import figure, output_notebook, show, gridplot, reset_output

import spykshrk.realtime.binary_record as bin_rec
import spykshrk.realtime.binary_record_cy as bin_rec_cy

pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.max_rows', 15)
pd.set_option('display.width', 180)

# output_notebook()

In [ ]:
rec_dir = '/opt/data36/daliu/realtime/spykshrk/bond_test'
#rec_dir = '/opt/data36/daliu/realtime/spykshrk/timing'
# rec_dir = '/home/daliu/output/test'
mpi_ranks = [0, 2, 3, 4, 5, 6]
file_prefix = 'test_animal'
manager_label = 'state'
file_postfix = 'bin_rec'

allrecs = {}

for mpi_rank in mpi_ranks:
    binrec = bin_rec_cy.BinaryRecordsFileReader(save_dir=rec_dir, file_prefix=file_prefix,
                                             mpi_rank=mpi_rank,
                                             manager_label=manager_label,
                                             file_postfix=file_postfix, filemeta_as_col=False)

    rec_pandas = binrec.convert_pandas()
    
    for rec_id, rec_pd in rec_pandas.items():
        existing_rec_pd = allrecs.setdefault(rec_id, pd.DataFrame())
        allrecs[rec_id] = existing_rec_pd.append(rec_pd, ignore_index=True)
    
for rec_id, rec_pd in allrecs.items():
    if 'timestamp' in rec_pd.columns:
        rec_pd.sort_values(['timestamp']).reset_index(drop=True)

In [3]:
rec_dir = '/opt/data36/daliu/realtime/spykshrk/bond04'
#rec_dir = '/opt/data36/daliu/realtime/spykshrk/timing'
# rec_dir = '/home/daliu/output/test'
mpi_rank = 2
file_prefix = 'test_animal'
manager_label = 'state'
file_postfix = 'bin_rec'

binrec = bin_rec_cy.BinaryRecordsFileReader(save_dir=rec_dir, file_prefix=file_prefix,
                                         mpi_rank=mpi_rank,
                                         manager_label=manager_label,
                                         file_postfix=file_postfix, filemeta_as_col=False)

cProfile.runctx('binrec.convert_pandas()', locals=locals(), globals=globals(), filename='pstats')

In [7]:
p = pstats.Stats('pstats')
p.strip_dirs()
p.sort_stats('cumulative')
p.print_stats()

In [5]:
p = pstats.Stats('pstats')
p.strip_dirs()
p.sort_stats('cumulative')
p.print_stats()

In [14]:
p.print_callees('binary_record.py:339')

In [3]:
allrecs.keys()

In [13]:
spike_dec = allrecs[4]
spike_dec

In [19]:
p1 = plt.plot(spike_dec[spike_dec['timestamp'] == 102141000].loc[:, 'x0':'x149'].values[0])

plt.show(p1)

In [4]:
decode_values = spike_dec.values[:,2:].astype('float')
decode_values = decode_values + 1e-200
decode_values = decode_values * 1/np.max(decode_values, axis=1)[:,None]

In [ ]:

plt.figure(figsize=[25,10])
im1 = plt.imshow(np.transpose(decode_values), aspect='auto')
plt.show(im1)